## Game Theory Task 2

###      
     Team Members:
     Ali Shaker - 20201701725      Diaa Amr - 20201701720        Rawan Mohsen - 20201701717      Mariam Alaa - 20201701739

In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter.ttk import Combobox
from PIL import ImageTk, Image
from tkinter.ttk import *

master = tk.Tk()
master.title("Task 2")
master.geometry('495x220')
master.resizable(False,False)

In [2]:
def find_best_responses(matrix_entries):
    row_max_second_indices = []
    col_max_first_indices = []
    best_responses = []

    # Loop over rows to find indices with maximum second number
    for i, row in enumerate(matrix_entries):
        max_second = max(int(cell.get().split(',')[1].strip()) for cell in row)
        row_max_second_indices.append([(i, j) for j, cell in enumerate(row) if int(cell.get().split(',')[1].strip()) == max_second])

    # Loop over columns to find indices with maximum first number
    for j in range(len(matrix_entries[0])):
        col_values = [int(matrix_entries[i][j].get().split(',')[0].strip()) for i in range(len(matrix_entries))]
        max_first = max(col_values)
        col_max_first_indices.append([(i, j) for i in range(len(matrix_entries)) if int(matrix_entries[i][j].get().split(',')[0].strip()) == max_first])

    # Find common indices in row_max_second_indices and col_max_first_indices
    for row_indices in row_max_second_indices:
        for col_indices in col_max_first_indices:
            intersection = list(set(row_indices) & set(col_indices))
            for index in intersection:
                best_responses.append(f"({index[0]+1},{index[1]+1}): {matrix_entries[index[0]][index[1]].get().strip()}")

    return best_responses

def find_maximin_moves(matrix_entries):
    player1_maxmin_moves = []
    player2_maxmin_moves = []

    # Loop over rows to find indices and values of minimum first number for player 1
    for i, row in enumerate(matrix_entries):
        min_first = min(int(cell.get().split(',')[0].strip()) for cell in row)
        player1_maxmin_moves.append((i, [int(cell.get().split(',')[0].strip()) for cell in row].index(min_first), min_first))

    # Loop over columns to find indices and values of minimum second number for player 2
    for j in range(len(matrix_entries[0])):
        col_values = [int(matrix_entries[i][j].get().split(',')[1].strip()) for i in range(len(matrix_entries))]
        min_second = min(col_values)
        player2_maxmin_moves.append((col_values.index(min_second), j, min_second))

    # Find the maximum value of player 1's maximin moves
    player1_maxmin_value = max(move[2] for move in player1_maxmin_moves)
    player1_maxmin_moves = [move[:2] for move in player1_maxmin_moves if move[2] == player1_maxmin_value]

    # Find the maximum value of player 2's maximin moves
    player2_maxmin_value = max(move[2] for move in player2_maxmin_moves)
    player2_maxmin_moves = [move[:2] for move in player2_maxmin_moves if move[2] == player2_maxmin_value]

    return player1_maxmin_value, player1_maxmin_moves, player2_maxmin_value, player2_maxmin_moves

def find_strategy_type(matrix_entries):
    num_rows = len(matrix_entries)
    num_cols = len(matrix_entries[0])
    
    # Check if all first numbers in a row are the same (pure strategy for Player 1)
    for i, row in enumerate(matrix_entries):
        first_numbers = [int(cell.get().split(',')[0].strip()) for cell in row]
        if len(set(first_numbers)) == 1:
            if all(number == first_numbers[0] for number in first_numbers):  # Ensure all first numbers are the same
                return "Pure Strategy for Player 1"  # Return "Player 1" 

    # Check if all second numbers in a column are the same (pure strategy for Player 2)
    for j in range(num_cols):
        column = [int(matrix_entries[i][j].get().split(',')[1].strip()) for i in range(num_rows)]
        if len(set(column)) == 1:
            if all(entry == column[0] for entry in column):  # Ensure all second numbers are the same
                return "Pure Strategy for Player 2"  # Return "Player 2" 

    # If no pure strategy is found, it's a mixed strategy
    return "Mixed Strategy", None

def eliminate_strictly_dominated(matrix_entries):
    while True:
        eliminated_rows = set()
        eliminated_cols = set()
        dominated_by_row = {}
        dominated_by_col = {}

        # Check domination by rows
        for i in range(len(matrix_entries)):
            if i in eliminated_rows:
                continue  # Skip already eliminated rows
            row_values = [int(cell.get().split(',')[0].strip()) for cell in matrix_entries[i]]

            # Compare with other rows
            for j in range(len(matrix_entries)):
                if j in eliminated_rows or i == j:
                    continue  # Skip already eliminated rows and self-comparison
                other_row_values = [int(cell.get().split(',')[0].strip()) for cell in matrix_entries[j]]

                # Check if one row dominates the other
                strictly_dominated = all(row_values[k] > other_row_values[k] for k in range(len(row_values)))
                if strictly_dominated:
                    eliminated_rows.add(j)
                    dominated_by_row[j] = i

        # Remove dominated rows
        matrix_entries = [row for i, row in enumerate(matrix_entries) if i not in eliminated_rows]

        # Check domination by columns
        for j in range(len(matrix_entries[0])):
            if j in eliminated_cols:
                continue  # Skip already eliminated columns
            col_values = [int(cell.get().split(',')[1].strip()) for cell in [row[j] for row in matrix_entries]]

            # Compare with other columns
            for k in range(len(matrix_entries[0])):
                if k in eliminated_cols or j == k:
                    continue  # Skip already eliminated columns and self-comparison
                other_col_values = [int(cell.get().split(',')[1].strip()) for cell in [row[k] for row in matrix_entries]]

                # Check if one column dominates the other
                strictly_dominated = all(col_values[i] > other_col_values[i] for i in range(len(col_values)))
                if strictly_dominated:
                    eliminated_cols.add(k)
                    dominated_by_col[k] = j

        # Remove dominated columns
        matrix_entries = [[cell for k, cell in enumerate(row) if k not in eliminated_cols] for row in matrix_entries]

        # If no more domination, break the loop
        if not eliminated_rows and not eliminated_cols:
            break

    # Display the final matrix
    domination_info = "The final matrix after domination is:\n"
    for row in matrix_entries:
        domination_info += ', '.join([cell.get().strip() for cell in row]) + '\n'

    # Print remaining undominated rows and columns
    undominated_rows = [i+1 for i in range(len(matrix_entries))]
    undominated_cols = [j+1 for j in range(len(matrix_entries[0]))]
    if undominated_rows or undominated_cols:
        if undominated_rows:
            domination_info += f"\nThe remaining undominated rows are: {undominated_rows}"
        if undominated_cols:
            domination_info += f"\nThe remaining undominated columns are: {undominated_cols}"
    else:
        domination_info += "\nNo possible domination."

    return domination_info

def eliminate_weakly_dominated(matrix_entries):
    while True:
        eliminated_rows = set()
        eliminated_cols = set()
        dominated_by_row = {}
        dominated_by_col = {}

        # Check weak domination by rows
        for i in range(len(matrix_entries)):
            if i in eliminated_rows:
                continue  # Skip already eliminated rows
            row_values = [int(cell.get().split(',')[0].strip()) for cell in matrix_entries[i]]

            # Compare with other rows
            for j in range(len(matrix_entries)):
                if j in eliminated_rows or i == j:
                    continue  # Skip already eliminated rows and self-comparison
                other_row_values = [int(cell.get().split(',')[0].strip()) for cell in matrix_entries[j]]

                # Check if one row weakly dominates the other
                weakly_dominated = all(row_values[k] >= other_row_values[k] for k in range(len(row_values)))
                if weakly_dominated and any(row_values[k] > other_row_values[k] for k in range(len(row_values))):
                    eliminated_rows.add(j)
                    dominated_by_row[j] = i

        # Remove dominated rows
        matrix_entries = [row for i, row in enumerate(matrix_entries) if i not in eliminated_rows]

        # Check weak domination by columns
        for j in range(len(matrix_entries[0])):
            if j in eliminated_cols:
                continue  # Skip already eliminated columns
            col_values = [int(cell.get().split(',')[1].strip()) for cell in [row[j] for row in matrix_entries]]

            # Compare with other columns
            for k in range(len(matrix_entries[0])):
                if k in eliminated_cols or j == k:
                    continue  # Skip already eliminated columns and self-comparison
                other_col_values = [int(cell.get().split(',')[1].strip()) for cell in [row[k] for row in matrix_entries]]

                # Check if one column weakly dominates the other
                weakly_dominated = all(col_values[i] >= other_col_values[i] for i in range(len(col_values)))
                if weakly_dominated and any(col_values[i] > other_col_values[i] for i in range(len(col_values))):
                    eliminated_cols.add(k)
                    dominated_by_col[k] = j

        # Remove dominated columns
        matrix_entries = [[cell for k, cell in enumerate(row) if k not in eliminated_cols] for row in matrix_entries]

        # If no more domination, break the loop
        if not eliminated_rows and not eliminated_cols:
            break

    # Display the final matrix
    domination_info = "The final matrix after weak domination is:\n"
    for row in matrix_entries:
        domination_info += ', '.join([cell.get().strip() for cell in row]) + '\n'

    # Print remaining undominated rows and columns
    undominated_rows = [i+1 for i in range(len(matrix_entries))]
    undominated_cols = [j+1 for j in range(len(matrix_entries[0]))]
    if undominated_rows or undominated_cols:
        if undominated_rows:
            domination_info += f"\nThe remaining undominated rows are: {undominated_rows}"
        if undominated_cols:
            domination_info += f"\nThe remaining undominated columns are: {undominated_cols}"
    else:
        domination_info += "\nNo possible weak domination."

    return domination_info

    

    

def generate_matrix():
    global matrix_frame, checkEntriesButton, matrix_entries, matrix_window

    game_type = gametype.get()
    if game_type not in (0, 1):
        messagebox.showerror("Error", "Please enter either 0 or 1 for the type of game.")
        return

    try:
        p1_moves = int(player1.get("1.0", "end-1c"))
        p2_moves = int(player2.get("1.0", "end-1c"))
        if p1_moves < 1 or p2_moves < 1:
            raise ValueError("Invalid number of moves")
    except ValueError as e:
        messagebox.showerror("Error", f"Number of moves for each player must be at least 1. {e}")
        return

    if matrix_window:
        matrix_window.destroy()  # Destroy the existing window if it exists

    matrix_window = tk.Toplevel(master)
    matrix_window.title("Matrix Entries")

    matrix_frame = tk.Frame(matrix_window)
    matrix_frame.pack(pady=20, padx=20)

    matrix_entries = []
    for i in range(p1_moves):
        row = []
        for j in range(p2_moves):
            tk.Label(matrix_frame, text=f"({i+1},{j+1}):", width=10).grid(row=i, column=2*j)
            entry = tk.Entry(matrix_frame, width=5)
            entry.grid(row=i, column=2*j+1)
            row.append(entry)
        matrix_entries.append(row)

    checkEntriesButton = tk.Button(matrix_window, text="Check Matrix Entries", command=check_matrix_entries, background="dark orange", fg="white")
    checkEntriesButton.pack(pady=10)

       

def check_matrix_entries():
    game_type = gametype.get()
    all_entries_valid = True

    for row in matrix_entries:
        for entry in row:
            value = entry.get().strip()
            if ',' not in value:
                messagebox.showerror("Error", "Please enter two numbers separated by a comma in each cell.")
                all_entries_valid = False
                break

            parts = value.split(',')
            if len(parts) != 2:
                messagebox.showerror("Error", "Please ensure only two numbers are entered in each cell.")
                all_entries_valid = False
                break

            try:
                num1, num2 = int(parts[0].strip()), int(parts[1].strip())
                if game_type == 0 and (num1 + num2 != 0):
                    messagebox.showerror("Error", "For ZeroSum game, the sum of each cell must be 0.")
                    all_entries_valid = False
                    break
            except ValueError:
                messagebox.showerror("Error", "Please enter valid integers in each cell.")
                all_entries_valid = False
                break

        if not all_entries_valid:
            break

    if all_entries_valid:
        best_responses = find_best_responses(matrix_entries)
        if best_responses:
            best_responses_text = '\n'.join(best_responses)
            best_responses_window = tk.Toplevel(master)
            best_responses_window.title("Game Analysis")
            best_responses_label = tk.Label(best_responses_window, text=f"Best response and Nash Equilibrium for the payoff matrix is:\n{best_responses_text}")
            best_responses_label.pack()

            maximin_result = find_maximin_moves(matrix_entries)
            player1_maxmin_value, player1_maxmin_moves, player2_maxmin_value, player2_maxmin_moves = maximin_result
            maximin_label = tk.Label(best_responses_window, text=f"Maximin Value for Player 1: {player1_maxmin_value}\nMaximin Value for Player 2: {player2_maxmin_value}")
            maximin_label.pack()

            # Calculate strategy type
            strategy_type = find_strategy_type(matrix_entries)
            
            # Display strategy type
            strategy_type_label = tk.Label(best_responses_window, text=f"Strategy Type: {strategy_type}")
            strategy_type_label.pack()

            # Eliminate dominated strategies
            strict_dominated_info = eliminate_strictly_dominated(matrix_entries)
            weak_dominated_info = eliminate_weakly_dominated(matrix_entries)

            # Display eliminated strategies
            strict_dominated_label = tk.Label(best_responses_window, text=f"\nStrictly Dominated Strategies:\n{strict_dominated_info}")
            strict_dominated_label.pack()

            weakly_dominated_label = tk.Label(best_responses_window, text=f"\nWeakly Dominated Strategies:\n{weak_dominated_info}")
            weakly_dominated_label.pack()
        else:
            no_best_response_label = tk.Label(best_responses_window, text="The payoff matrix has no best response or Nash Equilibirum.")
            no_best_response_label.pack()


# Main application window
master = tk.Tk()
master.title("Task 2")
master.geometry('495x220')  # Set the size of the main window
master.resizable(False, False)  # Prevent resizing of the window

# Labels and text inputs
tk.Label(master, text="Task 2", fg='dark orange', font=("Helvetica", 14)).pack()
tk.Label(master, text="Please enter the following", fg='black', font=("Helvetica", 10)).place(x=170, y=30)
tk.Label(master, text="Type of game (0 for ZeroSum, 1 for NonZeroSum)", fg='black', font=("Helvetica", 10)).place(x=100, y=60)

gametype = tk.IntVar()
tk.Entry(master, textvariable=gametype, fg='black', font=("Helvetica", 10), width=20).place(x=175, y=80)

tk.Label(master, text="# of moves for player 1", fg='black', font=("Helvetica", 10)).place(x=70, y=120)
player1 = tk.Text(master, fg='black', font=("Helvetica", 10), width=20, height=1)
player1.place(x=70, y=140)

tk.Label(master, text="# of moves for player 2", fg='black', font=("Helvetica", 10)).place(x=300, y=120)
player2 = tk.Text(master, fg='black', font=("Helvetica", 10), width=20, height=1)
player2.place(x=300, y=140)

# Button to generate matrix entries
generateMatrixButton = tk.Button(master, text="Confirm", background="dark orange", fg="white", command=generate_matrix)
generateMatrixButton.place(x=225, y=180)

matrix_frame = None
checkEntriesButton = None
matrix_window = None

In [3]:
master.mainloop()

: 